In [22]:
import tensorflow as tf
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from google.colab import drive
drive.mount('/content/drive')

# 전처리된 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/preprocessed_boiler_data.csv')

# 독립 변수(X)와 종속 변수(y) 설정
X = data.drop(columns=['Efficiency (Instantaneous)'])
y = data['Efficiency (Instantaneous)']

# Initialize StandardScaler
scaler = StandardScaler() # Create a StandardScaler object
X = scaler.fit_transform(X) # Fit and transform the data

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 최적 하이퍼파라미터 적용하여 랜덤 포레스트 모델 학습
best_params = {'n_estimators': 50, 'min_weight_fraction_leaf': 0.0, 'min_samples_split': 5,
               'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 25}
model = RandomForestRegressor(**best_params, random_state=42)
model.fit(X_train, y_train)

# 모델 평가
y_pred = model.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)

print(f"MAPE: {mape:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MSE: {mse:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MAPE: 7.9818
MAE: 0.0207
RMSE: 0.0473
MSE: 0.0022


In [23]:
import tensorflow as tf
import numpy as np
from sklearn.utils import shuffle

# 데이터 준비
X, y = shuffle(X, y, random_state=42)
y = y.values.reshape(-1, 1)

# 목표 변수(y)를 원래 StandardScaler로 스케일링
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)

# 신경망 모델 정의
model_dl = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # 보일러 효율 예측
])

model_dl.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                 loss='mse', metrics=['mse', 'mae'])

# 미니배치 학습
batch_size = 32
epochs = 3

history = model_dl.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.2, verbose=1)

# 최종 성능 평가
loss, mse, mae = model_dl.evaluate(X_test, y_test)
print(f"최종 MSE: {mse:.4f}")
print(f"최종 MAE: {mae:.4f}")

# 최적화 모델의 예측값 계산 및 원래 범위로 역변환
predicted_efficiency = model_dl.predict(X)
predicted_efficiency_original = scaler_y.inverse_transform(predicted_efficiency)

# 최적화된 보일러 효율값 계산 (역변환된 예측값의 평균)
optimized_efficiency = np.mean(predicted_efficiency_original)
print(f"최적화된 보일러 효율: {optimized_efficiency:.2f}%")

Epoch 1/3
70009/70009 [==============================] - 133s 2ms/step - loss: 0.0061 - mse: 0.0061 - mae: 0.0407 - val_loss: 0.0043 - val_mse: 0.0043 - val_mae: 0.0334
Epoch 2/3
70009/70009 [==============================] - 129s 2ms/step - loss: 0.0044 - mse: 0.0044 - mae: 0.0346 - val_loss: 0.0042 - val_mse: 0.0042 - val_mae: 0.0323
Epoch 3/3
17503/17503 [==============================] - 22s 1ms/step - loss: 0.0040 - mse: 0.0040 - mae: 0.0326
최종 MSE: 0.0040
최종 MAE: 0.0326
87512/87512 [==============================] - 101s 1ms/step
최적화된 보일러 효율: -0.00%
